# Advanced Applications of Mutate

## DRY and many transformations

In [19]:
import pandas as pd
from dfply import *
import matplotlib.pylab as plt
%matplotlib inline

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Exhibitions

In [20]:
dat_cols = ['ExhibitionBeginDate', 'ExhibitionEndDate', 'ConstituentBeginDate' ,'ConstituentEndDate']
exhibitions = pd.read_csv('./data/MoMAExhibitions1929to1989.csv', 
                          encoding="ISO-8859-1",
                          parse_dates=dat_cols)
exhibitions.head(2)

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionCitationDate,ExhibitionBeginDate,ExhibitionEndDate,ExhibitionSortOrder,ExhibitionURL,ExhibitionRole,ExhibitionRoleinPressRelease,...,Institution,Nationality,ConstituentBeginDate,ConstituentEndDate,ArtistBio,Gender,VIAFID,WikidataID,ULANID,ConstituentURL
0,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Curator,Director,...,NaN,American,1902,1981,"American, 19021981",Male,109252853.0,Q711362,500241556.0,moma.org/artists/9168
1,2557.0,1,"Cézanne, Gauguin, Seurat, Van Gogh","[MoMA Exh. #1, November 7-December 7, 1929]",1929-11-07,1929-12-07,1.0,moma.org/calendar/exhibitions/1767,Artist,Artist,...,NaN,French,1839,1906,"French, 18391906",Male,39374836.0,Q35548,500004793.0,moma.org/artists/1053


## The DRY principle

* DRY == *D*on't *R*epeat *Y*ourself
* Be sure to
    * Look for repeated patterns
    * Abstract the patterns

#### Example - Creating indicator columns

In [21]:
from collections import defaultdict
from composable import pipeable

make_indicator = pipeable(lambda lbl, val: 1 if lbl == val else 0)

(exhibitions
 >> select(X.Nationality)
 >> mutate(American = X.Nationality.map(make_indicator('American')))
) 

,Nationality,American
0,American,1
1,French,0
2,French,0
3,Dutch,0
4,French,0
...,...,...
34553,Japanese,0
34554,Japanese,0
34555,Japanese,0
34556,Japanese,0


## A WET transformation

This solution is WET, not DRY, as all 7 of the expressions in mutate are very similar ... if only here was a way to perform these similar expressions all at once ...

In [22]:
(exhibitions
 >> select(X.ExhibitionRole)
 >> mutate(Curator = X.ExhibitionRole.map(make_indicator('Curator')),
           Artist = X.ExhibitionRole.map(make_indicator('Artist')),
           Arranger = X.ExhibitionRole.map(make_indicator('Arranger')),
           Installer = X.ExhibitionRole.map(make_indicator('Installer')),
           CompetitionJudge = X.ExhibitionRole.map(make_indicator('Competition Judge')),
           Designer = X.ExhibitionRole.map(make_indicator('Designer')),
           Preparer = X.ExhibitionRole.map(make_indicator('Prepare')))
 >> head(5)
)
                                                    

,ExhibitionRole,Curator,Artist,Arranger,Installer,CompetitionJudge,Designer,Preparer
0,Curator,1,0,0,0,0,0,0
1,Artist,0,1,0,0,0,0,0
2,Artist,0,1,0,0,0,0,0
3,Artist,0,1,0,0,0,0,0
4,Artist,0,1,0,0,0,0,0


## Review - The `**` operators

Python functions use the `**` operator to 

1. **Pack** allow the capture of additional keyword arguments in function definitions.
2. **Unpack** unpack a `dict` of keyword arguments in function calls.

Note that these two actions are inverse operations.

#### Defining functions that capturing additional keyword arguments

In [23]:
def f(a, b = None, **kwargs):
    return f"a = {a}, b = {b}, added kw = {kwargs}"

In [24]:
f(2) #kwargs is empty by default.

'a = 2, b = None, added kw = {}'

In [25]:
# kwargs contains all extra kwargs when present.
f(2, Bob = "Tall", Alice = "Not tall") 

"a = 2, b = None, added kw = {'Bob': 'Tall', 'Alice': 'Not tall'}"

#### Unpacking a `dict` of kwargs

In [26]:
f(1, **{'Iverson':'Python', 'Bergen':'R', 'Malone':'Excel'})

"a = 1, b = None, added kw = {'Iverson': 'Python', 'Bergen': 'R', 'Malone': 'Excel'}"

# Applying many similar mutations.

To refactor many similar mutations

1. Create an expression `dict`
    * key: column name
    * value: `X.column` expression
2. Use mutate with `**` to unpack the expression `dict`

#### MoMA Artists

In [27]:
artists = pd.read_csv("./data/Artists.csv")
artists.head(2)

,ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
0,1,Robert Arneson,"American, 1930–1992",American,Male,1930,1992,NaN,NaN
1,2,Doroteo Arnaiz,"Spanish, born 1936",Spanish,Male,1936,0,NaN,NaN


#### MoMA Artwork

In [28]:
from more_dfply import fix_names

artwork = (pd.read_csv("./data/Artworks.csv")
           >> fix_names
           >> mutate(id = X.index + 1)
          )
artwork.head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Weight_kg,Width_cm,Seat_Height_cm,Duration_sec,id
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN,1
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN,2


#### Making the expression `dict`

In [29]:
from math import log, e
log1p = lambda num, base=e: log(num + 1, base)
log_columns = {'log{0}_plus_1'.format(base):X.Height_cm.apply(log1p, base = base)
               for base in (10, 2, e)}
log_columns 

{'log10_plus_1': <dfply.base.Intention at 0x7f91df28d350>,
 'log2_plus_1': <dfply.base.Intention at 0x7f91df28df50>,
 'log2.718281828459045_plus_1': <dfply.base.Intention at 0x7f91df28d910>}

#### Unpacking the expression in `mutate`

In [30]:
(artwork
 >> select(X.Height_cm)
 >> mutate(**log_columns)
 >> head(3)
)

,Height_cm,log10_plus_1,log2_plus_1,log2.718281828459045_plus_1
0,48.6000,1.695482,5.632268,3.903991
1,40.6401,1.619512,5.379902,3.729064
2,34.3000,1.547775,5.141596,3.563883


## Applying the same transformation on many different columns

Use `X['column']` if you need to reference columns

#### Define the column expressions

In [31]:
columns_to_log = contains('_cm').evaluate(artwork)
log_dimensions = {'log_{0}'.format(c):X[c].apply(log1p)
                  for c in (10, 2, e)}
log_dimensions

{'log_10': <dfply.base.Intention at 0x7f91df297690>,
 'log_2': <dfply.base.Intention at 0x7f91df297d50>,
 'log_2.718281828459045': <dfply.base.Intention at 0x7f91e9fce090>}

#### Unpack the expressions

In [32]:
(artwork
 >> select(contains('_cm'))
 >> mutate(**log_dimensions)
 >> head(3)
)

KeyError: 10

## Applying many similar mutations in `pyspark`

We can use the `dfply_spark.mutate` introduced in the last chapter.

## <font color="red"> Exercise 3.5.1 </font>

An **Indicator column** for a category contains 1 for the rows that match that label and 0 otherwise.  Again, create indicator columns for each of the categories in `exhibitions.ExhibitionRole`, but this time use the techniques in this section to make your solution more [DRY](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)

In [33]:
role_names = {"Curator","Artist","Arranger","Installer","CompetitionJudge","Designer","Preparer"}

In [39]:
make_indicator = pipeable(lambda lbl,val: 1 if lbl == val else 0)

In [42]:
columns_to_log = contains('Role').evaluate(exhibitions)
Role_columns = {'{0}'.format('Curator'):X.ExhibitionRole.map(make_indicator('Curator')),
                '{0}'.format('Artist'):X.ExhibitionRole.map(make_indicator('Artist')),
                '{0}'.format('Arranger'):X.ExhibitionRole.map(make_indicator('Arranger')),
                '{0}'.format('Installer'):X.ExhibitionRole.map(make_indicator('Installer')),
                '{0}'.format('CompetitionJudge'):X.ExhibitionRole.map(make_indicator('CompetitionJudge')),
                '{0}'.format('Designer'):X.ExhibitionRole.map(make_indicator('Designer')),
                '{0}'.format('Preparer'):X.ExhibitionRole.map(make_indicator('Preparer'))
               }
Role_columns

{'Curator': <dfply.base.Intention at 0x7f91d9997dd0>,
 'Artist': <dfply.base.Intention at 0x7f91d9987150>,
 'Arranger': <dfply.base.Intention at 0x7f91df2b0490>,
 'Installer': <dfply.base.Intention at 0x7f91df2b9c10>,
 'CompetitionJudge': <dfply.base.Intention at 0x7f91df2b9390>,
 'Designer': <dfply.base.Intention at 0x7f91df2b9810>,
 'Preparer': <dfply.base.Intention at 0x7f91e9cb1250>}

In [43]:
(exhibitions 
 >> select(X.ExhibitionRole)
 >> mutate(**Role_columns)
 >> head()
)

,ExhibitionRole,Curator,Artist,Arranger,Installer,CompetitionJudge,Designer,Preparer
0,Curator,1,0,0,0,0,0,0
1,Artist,0,1,0,0,0,0,0
2,Artist,0,1,0,0,0,0,0
3,Artist,0,1,0,0,0,0,0
4,Artist,0,1,0,0,0,0,0


## <font color="red"> Exercise 3.5.2</font>

We would like to convert all dimensions that are measured in cm to m.   To do this, perform the following tasks.

1. Create a regular expression match all dimension columns.  Use your RegEx to create a list of column names.
2. Create a `dict` with *new column names* as keys and column expressions that convert the corresponding column to m, where the expressions use `Intentions` to build the expressions.  **Hint:** Use a `dict` comprehension with an expression for the keys that changes `_cm` to `_m` in each of the old column names.  
3. Use the techniques shown in this section to apply these transformations using `mutate` and keyword unpacking to ensure your solution is DRY!

In [44]:
meter_change = lambda num: num/100

In [52]:
columns_to_change = contains('_cm').evaluate(artwork)
Meter_dimensions = {'{0}_meter'.format(c).replace("_cm",""):X[c].apply(meter_change)
                  for c in columns_to_change}
Meter_dimensions

{'Circumference_meter': <dfply.base.Intention at 0x7f256b9f0a10>,
 'Depth_meter': <dfply.base.Intention at 0x7f256b9f02d0>,
 'Diameter_meter': <dfply.base.Intention at 0x7f256b9e8cd0>,
 'Height_meter': <dfply.base.Intention at 0x7f256b9e8e90>,
 'Length_meter': <dfply.base.Intention at 0x7f256ba0f310>,
 'Width_meter': <dfply.base.Intention at 0x7f256ba0f610>,
 'Seat_Height_meter': <dfply.base.Intention at 0x7f256ba0f910>}

In [53]:
(artwork
 >> select(contains('_cm'))
 >> mutate(**Meter_dimensions)
 >> head()
)

,Circumference_cm,Depth_cm,Diameter_cm,Height_cm,Length_cm,Width_cm,Seat_Height_cm,Circumference_meter,Depth_meter,Diameter_meter,Height_meter,Length_meter,Width_meter,Seat_Height_meter
0,NaN,NaN,NaN,48.6000,NaN,168.9000,NaN,NaN,NaN,NaN,0.486000,NaN,1.689000,NaN
1,NaN,NaN,NaN,40.6401,NaN,29.8451,NaN,NaN,NaN,NaN,0.406401,NaN,0.298451,NaN
2,NaN,NaN,NaN,34.3000,NaN,31.8000,NaN,NaN,NaN,NaN,0.343000,NaN,0.318000,NaN
3,NaN,NaN,NaN,50.8000,NaN,50.8000,NaN,NaN,NaN,NaN,0.508000,NaN,0.508000,NaN
4,NaN,NaN,NaN,38.4000,NaN,19.1000,NaN,NaN,NaN,NaN,0.384000,NaN,0.191000,NaN
